![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_DistilBertForTokenClassification.ipynb)

# Import OpenVINO DistilBertForTokenClassification models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting DistilBertForTokenClassification models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for DistilBertForTokenClassification from DistilBertForTokenClassification   and they have to be in `Token Classification
` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00


[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [elastic/distilbert-base-cased-finetuned-conll03-english](https://huggingface.co/elastic/distilbert-base-cased-finetuned-conll03-english) model from HuggingFace as an example and load it as a `OVModelForTokenClassification`, representing an OpenVINO model.
- In addition to the OVModelForTokenClassification model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.intel import OVModelForTokenClassification
from transformers import AutoTokenizer

MODEL_NAME = "elastic/distilbert-base-cased-finetuned-conll03-english"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/954 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/257 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('ov_models/elastic/distilbert-base-cased-finetuned-conll03-english/tokenizer_config.json',
 'ov_models/elastic/distilbert-base-cased-finetuned-conll03-english/special_tokens_map.json',
 'ov_models/elastic/distilbert-base-cased-finetuned-conll03-english/vocab.txt',
 'ov_models/elastic/distilbert-base-cased-finetuned-conll03-english/added_tokens.json',
 'ov_models/elastic/distilbert-base-cased-finetuned-conll03-english/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [6]:
!ls -l {EXPORT_PATH}

total 255756
-rw-r--r-- 1 root root       960 Jun 21 20:43 config.json
-rw-r--r-- 1 root root 260795556 Jun 21 20:43 openvino_model.bin
-rw-r--r-- 1 root root    194823 Jun 21 20:43 openvino_model.xml
-rw-r--r-- 1 root root       125 Jun 21 20:43 special_tokens_map.json
-rw-r--r-- 1 root root      1279 Jun 21 20:43 tokenizer_config.json
-rw-r--r-- 1 root root    668923 Jun 21 20:43 tokenizer.json
-rw-r--r-- 1 root root    213450 Jun 21 20:43 vocab.txt


- As you can see, we need to move `vocab.txt` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [7]:
!mkdir -p {EXPORT_PATH}/assets

labels = ov_model.config.label2id
labels = sorted(labels, key=labels.get)

with open(f"{EXPORT_PATH}/assets/labels.txt", "w") as f:
    f.write("\n".join(labels))

!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets

In [9]:
!ls -lR {EXPORT_PATH}

ov_models/elastic/distilbert-base-cased-finetuned-conll03-english:
total 255548
drwxr-xr-x 2 root root      4096 Jun 21 21:25 assets
-rw-r--r-- 1 root root       960 Jun 21 20:43 config.json
-rw-r--r-- 1 root root 260795556 Jun 21 20:43 openvino_model.bin
-rw-r--r-- 1 root root    194823 Jun 21 20:43 openvino_model.xml
-rw-r--r-- 1 root root       125 Jun 21 20:43 special_tokens_map.json
-rw-r--r-- 1 root root      1279 Jun 21 20:43 tokenizer_config.json
-rw-r--r-- 1 root root    668923 Jun 21 20:43 tokenizer.json

ov_models/elastic/distilbert-base-cased-finetuned-conll03-english/assets:
total 216
-rw-r--r-- 1 root root     51 Jun 21 21:25 labels.txt
-rw-r--r-- 1 root root 213450 Jun 21 20:43 vocab.txt


In [8]:
!cat {EXPORT_PATH}/assets/labels.txt

O
B-PER
I-PER
B-ORG
I-ORG
B-LOC
I-LOC
B-MISC
I-MISC

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Import and Save DistilBertForTokenClassification in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [3]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 31.4 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [4]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `DistilBertForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [10]:
from sparknlp.annotator import DistilBertForTokenClassification

tokenClassifier = DistilBertForTokenClassification.loadSavedModel(
     EXPORT_PATH,
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your DistilBertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {EXPORT_PATH}_spark_nlp_openvino

total 254924
-rw-r--r-- 1 root root 261030482 Jun 21 21:27 distilbert_classification_openvino
drwxr-xr-x 4 root root      4096 Jun 21 21:27 fields
drwxr-xr-x 2 root root      4096 Jun 21 21:27 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DitilBertForTokenClassification model 😊

In [14]:
tokenClassifier_loaded = DistilBertForTokenClassification.load("./{}_spark_nlp_openvino".format(EXPORT_PATH))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [15]:
tokenClassifier_loaded.getClasses()

['B-LOC', 'I-ORG', 'I-MISC', 'I-LOC', 'I-PER', 'B-MISC', 'B-ORG', 'O', 'B-PER']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, NerConverter
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded,
    converter
])

example = spark.createDataFrame([
    ["Barack Obama was born in Hawaii and served as the 44th President of the United States."],
    ["Google was founded in September 1998 by Larry Page and Sergey Brin in California."],
    ["The Eiffel Tower is located in Paris and was completed in 1889."],
]).toDF("text")

model = pipeline.fit(example)
result = model.transform(example)

result.select("text", "ner.result").show(truncate=False)

result.selectExpr("explode(ner_chunk) as chunk").selectExpr(
    "chunk.result as text",
    "chunk.metadata['entity'] as entity"
).show(truncate=False)

+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------+
|text                                                                                  |result                                                                 |
+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------+
|Barack Obama was born in Hawaii and served as the 44th President of the United States.|[B-PER, I-PER, O, O, O, B-LOC, O, O, O, O, O, O, O, O, B-LOC, I-LOC, O]|
|Google was founded in September 1998 by Larry Page and Sergey Brin in California.     |[B-ORG, O, O, O, O, O, O, B-PER, I-PER, O, B-PER, I-PER, O, B-LOC, O]  |
|The Eiffel Tower is located in Paris and was completed in 1889.                       |[O, B-ORG, I-LOC, O, O, O, B-LOC, O, O, O, O, O, O]                    |
+---------------------------------

That's it! You can now go wild and use hundreds of `DistlBertForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
